# Generate response

# Question answering using a search API and re-ranking

Searching for relevant information can sometimes feel like looking for a needle in a haystack, but don’t despair, GPTs can actually do a lot of this work for us. In this guide we explore a way to augment existing search systems with various AI techniques, helping us sift through the noise.

Two ways of retrieving information for GPT are:

1. **Mimicking Human Browsing:** [GPT triggers a search](https://openai.com/blog/chatgpt-plugins#browsing), evaluates the results, and modifies the search query if necessary. It can also follow up on specific search results to form a chain of thought, much like a human user would do.
2. **Retrieval with Embeddings:** Calculate [embeddings](https://platform.openai.com/docs/guides/embeddings) for your content and a user query, and then [retrieve the content](Question_answering_using_embeddings.ipynb) most related as measured by cosine similarity. This technique is [used heavily](https://blog.google/products/search/search-language-understanding-bert/) by search engines like Google.

These approaches are both promising, but each has their shortcomings: the first one can be slow due to its iterative nature and the second one requires embedding your entire knowledge base in advance, continuously embedding new content and maintaining a vector database.

By combining these approaches, and drawing inspiration from [re-ranking](https://www.sbert.net/examples/applications/retrieve_rerank/README.html) methods, we identify an approach that sits in the middle. **This approach can be implemented on top of any existing search system, like the Slack search API, or an internal ElasticSearch instance with private data**. Here’s how it works:

![search_augmented_by_query_generation_and_embeddings_reranking.png](../images/search_rerank_answer.png)

**Step 1: Search**

1.  User asks a question.
2.  GPT generates a list of potential queries.
3.  Search queries are executed in parallel.

**Step 2: Re-rank**

1.  Embeddings for each result are used to calculate semantic similarity to a generated hypothetical ideal answer to the user question.
2.  Results are ranked and filtered based on this similarity metric.

**Step 3: Answer**

1.  Given the top search results, the model generates an answer to the user’s question, including references and links.

This hybrid approach offers relatively low latency and can be integrated into any existing search endpoint, without requiring the upkeep of a vector database. Let's dive into it! We will use the [News API](https://newsapi.org/) as an example domain to search over.

## Setup

In addition to your `OPENAI_API_KEY`, you'll have to include a `NEWS_API_KEY` in your environment. You can get an API key [here](https://newsapi.org/).


NameError: name 'openai' is not defined

In [4]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython import display  # for pretty printing
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import openai  # for using GPT and getting embeddings
import os  # for loading environment variables
import requests  # for making the API requests
from tqdm.notebook import tqdm  # for printing progress bars

# Load environment variables


GPT_MODEL = "gpt-3.5-turbo"


# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input): 
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]
from dotenv import load_dotenv
load_dotenv()
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
news_api_key = os.getenv("NEWS_API_KEY")


## 1. Search

It all starts with a user question.


In [5]:
# User asks a question
# load the question
file_path = "data/final_transcript.txt"
with open(file_path, "r") as f:
    final_transcript = f.read()
USER_QUESTION = final_transcript
print(USER_QUESTION)

Good afternoon, everyone. And welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a stellar second quarter (Q2) with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from our scalable business model. Our Earnings Before Interest, Taxes, Depreciation, and Amortization (EBITDA) has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially thanks to the expansion of our high yield savings product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in Collateralized Debt Obligations (CDOs), and Residential Mortgage-Backed Securities (RMBS). We've also invested $25 million in AAA rated corporate 

Now, in order to be as exhaustive as possible, we use the model to generate a list of diverse queries based on this question.


In [6]:
QUERIES_INPUT = f"""
You have access to a search API that returns recent news articles.
Generate an array of search queries that are relevant to this question.
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms.
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2'].
Be creative. The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

# Let's include the original question as well for good measure
queries.append(USER_QUESTION)

queries

['FinTech Plus Sync second quarter 2023 earnings call',
 'FinTech Plus Q2 revenue',
 'FinTech Plus Q2 gross profit margin',
 'FinTech Plus Q2 EBITDA',
 'FinTech Plus Q2 net income',
 'FinTech Plus high yield savings product line expansion',
 'FinTech Plus RoboAdvisor platform',
 'FinTech Plus asset-backed securities portfolio diversification',
 'FinTech Plus CDOs investment',
 'FinTech Plus RMBS investment',
 'FinTech Plus corporate bonds investment',
 'FinTech Plus balance sheet',
 'FinTech Plus debt-to-equity ratio',
 'FinTech Plus organic user growth',
 'FinTech Plus CAC',
 'FinTech Plus LTV',
 'FinTech Plus LTVCAC ratio',
 'FinTech Plus risk management',
 'FinTech Plus VaR model',
 'FinTech Plus Tier 1 Capital Ratio',
 'FinTech Plus Q3 forecast',
 'FinTech Plus blockchain solutions',
 'FinTech Plus AI-driven predictive analytics',
 'FinTech Plus IPO',
 'FinTech Plus Q3 expectations',
 "Good afternoon, everyone. And welcome to FinTech Plus Sync's second quarter 2023 earnings call. I

The queries look good, so let's run the searches.


In [9]:
def search_news(
    query: str,
    news_api_key: str = news_api_key,
    num_articles: int = 50,
    from_datetime: str = "2023-09-12",  # the 2023 NBA finals were played in June 2023
    to_datetime: str = "2023-10-11",
) -> dict:
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    if len(query) > 499:
        continue
    result = search_news(query)
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

# remove duplicates
articles = list({article["url"]: article for article in articles}.values())

print("Total number of articles:", len(articles))
print("Top 5 articles of query 1:", "\n")

for article in articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print()


  0%|          | 0/26 [00:00<?, ?it/s]

Total number of articles: 51
Top 5 articles of query 1: 

Title: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript
Description: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript September 12, 2023 Edgio, Inc. misses on earnings expectations. Reported EPS is $-0.12 EPS...
Content: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript September 12, 2023
Edgio, Inc. misses on...

Title: IBEX Limited (NASDAQ:IBEX) Q4 2023 Earnings Call Transcript
Description: IBEX Limited (NASDAQ:IBEX) Q4 2023 Earnings Call Transcript September 13, 2023 IBEX Limited misses on earnings expectations. Reported EPS is $0.33 EPS...
Content: IBEX Limited (NASDAQ:IBEX) Q4 2023 Earnings Call Transcript September 13, 2023
IBEX Limited misses ...

Title: Marketing for Coaches: How to Win Coaching Clients + Real-Life Hints from Coaching Experts
Description: OK, you already defined your coaching niche and your target audience. You also checked off arranging your coaching services into clear pack

As we can see, oftentimes, the search queries will return a large number of results, many of which are not relevant to the original question asked by the user. In order to improve the quality of the final answer, we use embeddings to re-rank and filter the results.

## 2. Re-rank

Drawing inspiration from [HyDE (Gao et al.)](https://arxiv.org/abs/2212.10496), we first generate a hypothetical ideal answer to rerank our compare our results against. This helps prioritize results that look like good answers, rather than those similar to our question. Here’s the prompt we use to generate our hypothetical answer.


In [10]:
HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer will be used to rank search results. 
Pretend you have all the information you need to answer, but don't use any actual facts. Instead, use placeholders
like NAME did something, or NAME said something at PLACE. 

User question: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer


"Good afternoon, everyone. And welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a remarkable second quarter (Q2) with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from our scalable business model. Our Earnings Before Interest, Taxes, Depreciation, and Amortization (EBITDA) has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially thanks to the expansion of our high yield savings product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in Collateralized Debt Obligations (CDOs), and Residential Mortgage-Backed Securities (RMBS). We've also invested $25 million in AAA rated corpor

Now, let's generate embeddings for the search results and the hypothetical answer. We then calculate the cosine distance between these embeddings, giving us a semantic similarity metric. Note that we can simply calculate the dot product in lieu of doing a full cosine similarity calculation since the OpenAI embeddings are returned normalized in our API.


In [11]:
hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
article_embeddings = embeddings(
    [
        f"{article['title']} {article['description']} {article['content'][0:100]}"
        for article in articles
    ]
)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding))

cosine_similarities[0:10]


[0.8036852724546562,
 0.7973797667134029,
 0.6907574075157661,
 0.7778821397555651,
 0.7731251314563298,
 0.7613239566298422,
 0.7811087094463756,
 0.7380903967660174,
 0.7090377977189425,
 0.7846978859204605]

Finally, we use these similarity scores to sort and filter the results.


In [12]:
scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

# Print top 5 articles
print("Top 5 articles:", "\n")

for article, score in sorted_articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()


Top 5 articles: 

Title: We eventually want to become a reinsurance broker: Yashish Dahiya, PB Infotech
Description: “This year we might be performing a little better than expected, but that is not anything significant right. It is just maybe 5-10% better than expected. So, our guidance stays the same. This year we will broadly be a PAT positive and every year in our core b…
Content: Yashish Dahiya, Chairman &amp; CEO, PB Fintech, says we have enough data, which implies that we beli...
Score: 0.809970402084174

Title: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript
Description: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript September 12, 2023 Edgio, Inc. misses on earnings expectations. Reported EPS is $-0.12 EPS...
Content: Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript September 12, 2023
Edgio, Inc. misses on...
Score: 0.8036852724546562

Title: CIO Talks - Banking and Financial solutions. Fintech. Analytics Miercuri, 20 Septembrie 2023, începand cu

Awesome! These results look a lot more relevant to our original query. Now, let's use the top 5 results to generate a final answer.

## 3. Answer


In [13]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. Reference the relevant search result urls as markdown links.
"""

completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=0.5,
    stream=True,
)

text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))

Based on the information provided in the question, FinTech Plus had a stellar second quarter (Q2) in 2023. The company reported a revenue of $125 million, which represents a 25% increase year over year. The gross profit margin stood at a solid 58%, thanks to cost efficiencies gained from the scalable business model. Earnings Before Interest, Taxes, Depreciation, and Amortization (EBITDA) surged to $37.5 million, resulting in a remarkable 30% EBITDA margin. Net income for the quarter rose to $16 million, a noteworthy increase from $10 million in Q2 2022.

FinTech Plus has experienced substantial growth in its total addressable market by expanding its high yield savings product line and introducing a new RoboAdvisor platform. The company has also diversified its asset-backed securities portfolio by investing heavily in Collateralized Debt Obligations (CDOs) and Residential Mortgage-Backed Securities (RMBS). Additionally, FinTech Plus has invested $25 million in AAA rated corporate bonds to enhance risk-adjusted returns.

In terms of financial position, FinTech Plus has total assets of $1.5 billion and total liabilities of $900 million, resulting in a solid equity base of $600 million. The debt-to-equity ratio stands at a healthy figure of 1.5, considering the company's expansionary phase.

FinTech Plus has seen substantial organic user growth, with a 15% decrease in Customer Acquisition Cost (CAC) and a 25% increase in Lifetime Value (LTV). The LTV to CAC (LTVCAC) ratio is an impressive 3.5%.

The company has implemented a Value at Risk (VaR) model with a 99% confidence level, indicating that the maximum loss will not exceed $5 million in the next trading day. FinTech Plus follows a conservative approach to leverage management and maintains a healthy Tier 1 Capital Ratio of 12.5%.

Looking ahead, FinTech Plus forecasts a positive third quarter (Q3) with expected revenue of around $135 million and 8% quarter over quarter growth. This growth will be primarily driven by the company's cutting-edge blockchain solutions and AI-driven predictive analytics. FinTech Plus is also excited about the upcoming Initial Public Offering (IPO) of its subsidiary, Pay Plus, which is expected to raise $200 million, significantly enhancing liquidity and enabling aggressive growth strategies.

For more information, you can refer to the following sources:

- [We eventually want to become a reinsurance broker: Yashish Dahiya, PB Infotech](https://economictimes.indiatimes.com/markets/expert-view/we-eventually-want-to-become-a-reinsurance-broker-yashish-dahiya-pb-infotech/articleshow/103925028.cms)
- [Edgio, Inc. (NASDAQ:EGIO) Q2 2023 Earnings Call Transcript](https://finance.yahoo.com/news/edgio-inc-nasdaq-egio-q2-125307025.html)
- [CIO Talks - Banking and Financial solutions. Fintech. Analytics Miercuri, 20 Septembrie 2023, începand cu orele 14:00](https://www.wall-street.ro/articol/Careers/300810/cio-talks-banking-and-financial-solutions-fintech-analytics-miercuri-20-septembrie-2023-incepand-cu-orele-14-00.html)
- [IBEX Limited (NASDAQ:IBEX) Q4 2023 Earnings Call Transcript](https://finance.yahoo.com/news/ibex-limited-nasdaq-ibex-q4-120909945.html)
- [FinTech IPO Index Slides 3% as Affirm and Blend Lead Declining Stocks](https://www.pymnts.com/news/fintech-investments/2023/fintech-ipo-index-slides-3-pct-affirm-blend-lead-declining-stocks/)

APIError: HTTP code 200 from API ("{\"rate_limit_usage\": {\)